## Partiționare

La crearea unui RDD se poate specifica numărul de partiții
<br>Numărul implicit este numărul de workers definit la crearea lui `SparkContext`

In [ ]:
from pyspark import SparkContext

### Crearea unui `SparkContext` cu 2 workers

In [ ]:
sc = SparkContext(master="local[2]")

In [ ]:
#Paralelizati lista primelor 1000000 numere intregi
# A =

### Utilizarea `getNumPartitions` pentru regăsirea numărului de partiții create

In [ ]:
print(A.getNumPartitions())

### Putem repartiționa _A_ în orice număr de partiții

In [ ]:
D = A.repartition(10)

In [ ]:
print(D.getNumPartitions())

### Putem stabili numărul de partiții la crearea RDD-ului cu argumentul `numSlices`

In [ ]:
A = sc.parallelize(range(1000000),numSlices=8)

In [ ]:
#Afisati numarul de partitii


### De ce sunt importante partițiile?

* Definesc unitatea asupra căreia lucrează executorul
* Ar trebui să existe tot atâtea partiții cât numărul de noduri worker
* Partițiile mai mici pot permite mai mult paralelism

## Repartiționare pentru Load Balancing

Presupunem că pornim cu 10 partiții, toate cu exact același număr de elemente

In [ ]:
A=sc.parallelize(range(1000000)).map(lambda x:(x,x)).partitionBy(10)
print(A.glom().map(len).collect())

Presupunem că dorim să folosim funcția **`filter()`** pentru a selecta o parte din elementele lui A.<br>
În unele partiții pot rămâne mai multe elemente decât în altele.

In [ ]:
#Selectati 20% din elemente, pe baza unui filtru care retine doar numerele divizibile cu 5
B=
# Obtineti numarul de elemente din partitiile lui B


Operațiile viitoare asupra lui B vor folosi doar 2 workers.<br>
Celalți workers nu vor face nimic, deoarece partițiile lor sunt vide.

### Pentru a corecta această situație trebuie să repartiționăm RDD-ul neechilibrat. <br>Un mod de a realiza acest lucru este să repartiționăm folosind o cheie nouă, cu ajutorul metodei `partitionBy()`

* Metoda **`.partitionBy(k)`** așteaptă un RDD de forma **`(key,value)`**, unde cheile sunt întregi.
* Partiționează RDD-ul în **`k`** partiții.
* Elementul **`(key,value)`** este plasat în partiția **`key % k`**

In [ ]:
#Redistribuiți B în 10 partiții cu număr egal de elemente
#C=
#Obțineți numărul de elemente din partițiile lui C

Observăm că **`C`** constă din doar 200,000 elemente din partiția neechilibrată **`B`** dar le distribuie în partiții egale de câte 20,000 elemente.

### O altă abordare: utilizarea partiționării aleatoare cu ajutorul **`repartition(k)`**
* Un **avantaj** al partiționării aleatoare este că aceasta nu necesită definirea unei chei.
* Un **dezavantaj** al partitionării aleatoare este că nu există control asupra acesteia (nu putem stabili în ce partiții merg elementele).

In [ ]:
C=B.repartition(10)
print(C.glom().map(len).collect())

## `Glom()`
* În general, Spark nu permite worker-ului să refere elemente specifice din RDD.
* Aceasta poate fi o limitare majoră.

#### `glom()` transformă fiecare partiție într-o listă de elemente.<br> Creează un RDD de liste, câte o listă pentru fiecare partiție.<br>Worker-ii pot referi elemente ale partiției prin indexul acestora, dar nu pot fi atribuite valori elementelor, RDD-ul fiind imutabil.

* Considerăm **comanda folosită anterior pentru a număra elementele fiecărei partiții**: `print(C.glom().map(len).collect())`
* Am folosit `glom()` ca să transformăm fiecare partiție într-o listă.
* Am folosit `len` asupra fiecărei partiții pentru a obține lungimea listei - dimensiunea partiției.
* Am aplicat `collect` pentru a colecta rezultatele și a le afișa.

### Un exemplu mai complex
Metoda `glom()` poate fi utilă în mai multe cazuri.
<br>
De exemplu, presupunem că dorim să obținem primul element, numărul de elemente și suma dintre primul și ultimul element din partițiile neechilibrate pe care le-am obținut din `A` în `B`. Dacă o partiție este vidă returnăm `None`.

In [ ]:
# Definiți funcția ce returnează tuplul de 3 elemente cerut pentru fiecare partiție
#def getPartitionInfo(P):
 
#Apelați funcția pentru partițiile lui B și afișați rezultatul


In [ ]:
sc.stop()